## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-10-02-16-21 +0000,nyt,Middle East Crisis Updates: Israel Approves Ga...,https://www.nytimes.com/live/2025/10/09/world/...
1,2025-10-10-02-15-52 +0000,nyt,"Iran, a Longtime Hamas Backer, Signals Support...",https://www.nytimes.com/2025/10/09/world/europ...
2,2025-10-10-02-14-14 +0000,nyt,Powerful Earthquake Strikes Southern Philippines,https://www.nytimes.com/2025/10/09/world/asia/...
3,2025-10-10-02-13-44 +0000,nypost,NYC Council approves bill to boost testing for...,https://nypost.com/2025/10/09/us-news/council-...
4,2025-10-10-02-13-37 +0000,nypost,Family asks for ‘Halloween miracle’ after thie...,https://nypost.com/2025/10/09/us-news/family-a...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2318/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
48,trump,52
5,gaza,39
6,deal,37
225,new,21
3,israel,20


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
114,2025-10-09-20-54-59 +0000,nypost,Jubilant Trump vows Gaza peace deal will ‘move...,https://nypost.com/2025/10/09/us-news/jubilant...,194
159,2025-10-09-19-02-29 +0000,nypost,Rep. Andy Barr latest to nominate Trump for No...,https://nypost.com/2025/10/09/us-news/rep-andy...,173
107,2025-10-09-21-17-33 +0000,nypost,Zohran Mamdani refuses to credit Trump over Ga...,https://nypost.com/2025/10/09/us-news/zohran-m...,164
84,2025-10-09-22-13-20 +0000,nypost,Obama applauds Trump’s Gaza peace deal: ‘We sh...,https://nypost.com/2025/10/09/us-news/obama-ap...,154
270,2025-10-09-11-51-45 +0000,nypost,Gazans join Israelis in chanting Trump’s name ...,https://nypost.com/2025/10/09/us-news/gazans-j...,151


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
114,194,2025-10-09-20-54-59 +0000,nypost,Jubilant Trump vows Gaza peace deal will ‘move...,https://nypost.com/2025/10/09/us-news/jubilant...
66,83,2025-10-09-22-51-00 +0000,wsj,The Justice Department has secured an indictme...,https://www.wsj.com/politics/justice-departmen...
334,67,2025-10-09-04-23-07 +0000,nyt,President Trump Says Israel and Hamas Agree to...,https://www.nytimes.com/2025/10/08/world/middl...
243,53,2025-10-09-13-21-56 +0000,nypost,Trump tells The Post he’ll visit the Middle Ea...,https://nypost.com/2025/10/09/us-news/trump-te...
14,53,2025-10-10-01-15-44 +0000,nyt,Federal Judge Temporarily Blocks Trump’s Natio...,https://www.nytimes.com/2025/10/09/us/judge-na...
284,47,2025-10-09-10-39-00 +0000,wsj,China introduced new controls on rare-earth ex...,https://www.wsj.com/economy/trade/china-impose...
150,40,2025-10-09-19-35-40 +0000,nypost,Chuck Schumer ripped for claiming shutdown is ...,https://nypost.com/2025/10/09/us-news/chuck-sc...
220,37,2025-10-09-15-18-12 +0000,latimes,Older men across L.A. County thought they had ...,https://www.latimes.com/california/story/2025-...
42,34,2025-10-09-23-54-00 +0000,wsj,The Bureau of Labor Statistics will bring some...,https://www.wsj.com/politics/policy/federal-go...
155,31,2025-10-09-19-23-47 +0000,nyt,Texas High Court Halts Execution of Robert Rob...,https://www.nytimes.com/2025/10/09/us/texas-sh...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
